# Введение
В соревновании была задача предсказать популярность объявления о продаже дома. Тут я попытаюсь рассказать о некоторых методах, которые применялись участниками и посмотреть, какие из них дали наибольший результат.

В ходе этого ноутбука мы:
* исследуем данные
* создадим много новых фич
* проанализируем важность фич с помощью shap

В конце есть интересный вывод!

References

При работе с геоданными, вдохновился [этим ноутбуком](https://www.kaggle.com/gaborfodor/from-eda-to-the-top-lb-0-367?scriptVersionId=1515342) от Белуги, грандмастера на Каггле

## Загрузка библиотек и вспомогательных функций

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [ ]:
train = pd.read_csv('../input/gusigagaga/train-2.csv', sep = ',')
test = pd.read_csv('../input/gusigagaga/test-2.csv', sep = ',')

In [ ]:
X_train = train.drop(['TARGET'], axis = 1)
y_train = train['TARGET']
X_test = test

In [ ]:
X_train.head(3)

In [ ]:
X_train.describe()

Как улучшить бейзлайн? 
- created. Со временем популярность любого объявления снижается. 
- features. Сделать отдельную колонку под каждую фичу. 
- latitude, longitude, street_address. Посмотреть на карте, где расположены самые популярные здания.






## Стратегия для валидации
Сначала посмотрим на различия train и test.

In [ ]:
print(X_train.created.min())
print(X_test.created.min())

print(X_train.created.max())
print(X_test.created.max())

In [ ]:
city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)
fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True)
N = 10000
ax[0].scatter(X_train['longitude'].values[:N], X_train['latitude'].values[:N],
              color='blue', s=1, label='train', alpha=0.1)
ax[1].scatter(X_test['longitude'].values[:N], X_test['latitude'].values[:N],
              color='green', s=1, label='test', alpha=0.1)
fig.suptitle('Train and test area complete overlap.')
ax[0].legend(loc=0)
ax[0].set_ylabel('latitude')
ax[0].set_xlabel('longitude')
ax[1].set_xlabel('longitude')
ax[1].legend(loc=0)
plt.ylim(city_lat_border)
plt.xlim(city_long_border)
plt.show()

В этом случае разделение на train и test было случайнымпо геоданным, это позволит нам использовать обучение без учителя и feature extraction на всем датасете

## Feature engineering

Чтобы модель искала нелинейные зависимости добавим функции от признаков и полиномиальные признаки:

In [ ]:
from itertools import combinations
def add_functions_and_polynoms(X):
    X = X.copy()
    columns_to_poly = list(X.select_dtypes(include=np.number).columns)
    for combination in combinations(columns_to_poly, 2):
        X[f'poly_{combination[0]}_{combination[1]}'] = X[combination[0]] * X[combination[1]]
    for column in columns_to_poly:
        X[f'x**2_{column}'] = X[column] ** 2
        X[f'log_{column}'] = np.log(X[column])
        X[f'sqrt_{column}'] = np.sqrt(X[column])
        X[f'1 / {column}'] = 1 / X[column]
        X[f'sin_{column}'] = np.sin(X[column])
        X[f'cos_{column}'] = np.cos(X[column])
        X[f'sin^2_{column}'] = np.sin(X[column]) ** 2
        X[f'cos^2_{column}'] = np.cos(X[column]) ** 2
    X = X.replace(np.nan, 0)
    X = X.replace(np.inf, 0)
    X = X.replace(-np.inf, 0)
    return X
X_train = add_functions_and_polynoms(X_train)
X_test = add_functions_and_polynoms(X_test)

Тут пытаюсь разобраться с колонкой features. Делаем one-hot encoding, чтобы по каждому объявлению мы знали, например, есть ли в этом доме лифт или нет

In [ ]:
listy = list(X_test.columns)

def add_features_onehot(X):
    X = X.copy()
    X['features'] = X.features.apply(lambda x: x[1:-1].lower().replace("'", "").replace('"', "").split(', '))
    
    X = X.join(X.features.str.join('|').str.get_dummies())
    return X

X_train = add_features_onehot(X_train)
X_test = add_features_onehot(X_test)


из признака features оставим только те признаки, которые есть как в train, так и test. При One-hot encoding было выяснилось, что в колонке features люди добавляли много уникальных значений, которые никак не помогут модели.

In [ ]:
X_test[X_test.columns.intersection(X_train.columns)].columns

вот так красиво теперь выглядит признак features

### работа с геоданными
Чтобы обработать геоданные, я использовал PCA и Agglomerative clustering.
Мы используем PCA для преобразования координат долготы и широты. В данном случае речь не идет об уменьшении размерности, поскольку мы преобразуем 2D-> 2D. Ротация может помочь при расщеплении дерева решений в catboost.


In [ ]:
from sklearn.decomposition import PCA

coords = np.vstack((X_train[['latitude', 'longitude']].values, X_test[['latitude', 'longitude']].values))
pca = PCA().fit(coords)

def add_PCA_i(X, i):
    X = X.copy()
    X[f'pca{i}'] = pca.transform(X[['latitude', 'longitude']])[:, i]
    return X

X_train = add_PCA_i(X_train, 0)
X_test = add_PCA_i(X_test, 0)

AgglomerativeClustering используем для обычного разделения точек на плоскости

In [ ]:


#train.TARGET = pd.factorize(train['TARGET'])[0] + 1

from sklearn.cluster import AgglomerativeClustering

# creates 40 clusters using hierarchical clustering.
i = 40
agc = AgglomerativeClustering(n_clusters =i, affinity='euclidean', linkage='ward')

def add_agglomerative_clusters(X):
    X = X.copy()
    X['cluster'] = np.where(((X.longitude > -73.775) |
                             (X.longitude < -74.050) |
                             (X.latitude > 40.930)   |
                             (X.latitude < 40.55)), i, agc.fit_predict(X[['latitude','longitude']]))
    X = X.join(pd.get_dummies(X.cluster))
    return X

X_train = add_agglomerative_clusters(X_train)
X_test = add_agglomerative_clusters(X_test)


### Житейская мудрость - price_per_bedroom, считаем популярность риэлтора и building_id

In [ ]:
def wordlinnes(X):
    X = X.copy()
    X['price_per_bedroom'] = X["price"] / X["bedrooms"]
    X["price_per_bathroom"] = X["price"] / X["bathrooms"]
    X = X.drop(['bedrooms','bathrooms'], axis=1)
    return X
X_train = wordlinnes(X_train)
X_test = wordlinnes(X_test)


Обрабатываем building_id и manager_id, они дали высокое влияние на выход модели.С manager_id это можно логически объяснить - хороший риэлтор умеет правильно подкрутить объявление так,чтобы оно было популярным. А вот почему listing_id дает буст в скоре - непонятно, все-таки в этом признаке все элементы уникальные.

In [ ]:
building_ids = X_train['building_id'].value_counts()
manager_ids = X_train['manager_id'].value_counts()
def countsy(X):
    X = X.copy()
    X['manager_ids_count'] = X['manager_id'].apply(lambda x: manager_ids[x] if x in manager_ids else 0)
    X['building_ids_count'] = X['building_id'].apply(lambda x: building_ids[x] if x in building_ids else 0)
    return X
X_train = countsy(X_train)
X_test = countsy(X_test)

### Время

In [ ]:
def timelines(X):
    X["created"] = X["created"].astype("datetime64")
    X['Weekday'] = X.created.dt.weekday
    X['day_of_month'] = X.created.dt.day
    X['hour'] = X.created.dt.hour
    X['is_weekend'] = X.created.apply(lambda x: 1 if x.date().weekday() in (5, 6) else 0)
    X['month'] = X.created.dt.month
    X['week'] = X.created.dt.week

    X['hour_weekofyear'] = X.created.dt.weekofyear
    X['minute'] = X['created'].dt.minute
    X['pickup_week_hour'] = X['Weekday'] * 24 + X['hour']

    basedate = pd.Timestamp('2016-06-29 18:30:41')
    X['days_since_last'] = X.created.apply(lambda x: (basedate - x).days)
    return X
X_train = timelines(X_train)
X_test = timelines(X_test)

In [ ]:
test1 = X_test[X_test.columns.intersection(X_train.columns)]
train1 = X_train[X_train.columns.intersection(X_test.columns)].merge(train.TARGET, left_index=True, right_index=True)

feats_list1 = ['price'] + list(test1[test1.columns.intersection(train1.columns)].columns)[15:]

In [ ]:
from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_class_weight


X_train = train1.loc[:, feats_list1]
X_test = test1.loc[:, feats_list1]

y_train = train1.loc[:, 'TARGET'].values

classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

model = CatBoostClassifier(class_weights=class_weights, loss_function='MultiClass', 
                           eval_metric='Accuracy', custom_loss='Accuracy', verbose = 250)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
import shap
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(Pool(X_train, y_train))
shap.summary_plot(shap_values, X_train, plot_type="bar")

Тут изображен топ признаков, которые больше всего повлияли на модель. В топе по влиянию оказались перемноженные признаки, производные от признака features, pca0 повлиял сравнительно неплохо, но AGC никак не повлиял. Отдельно можно заметить, что если указано, что в объявлении есть hardwood floors (паркетные полы), то это значительно влияет на популярность объявления, потому что class 2 имеет самую большую относительную долю в этом признаке.
Теперь посмотрим, как эти признаки повлияли на модель: положительно или отрицательно.

In [ ]:
shap.summary_plot(shap_values[1], X_train)

На приведенном ниже графике объекты сортируются по сумме величин значений SHAP по всем выборкам и используются значения SHAP, чтобы показать распределение влияний каждого объекта на выходные данные модели. Цвет представляет значение функции (красный высокий, синий низкий). Это показывает, например, что высокий признак no fee (1 - есть плата, 0 - нет платы) снижает прогнозируемую популярность объявления. Отдельно стоить заметить,что price_per_bedroom только мешает модели, этот признак не дает никакой информации о том, какой класс нужно дать конкретному элементу. Фича из житейской мудрости слишком коррелирует с признаком price.

## Основной вывод:
основной буст дали нелогичные фичи: poly_listing_id_price - это просто перемноженные listing_id и price. В чем вообще физический смысл этого признака? А в чем смысла признака pca_0? Почему listing_id вообще имеет высокое влияние, если каждое значение этого признака уникальное? Почему логичные признаки типа hour, Это всё мы ведем к тому, что на таких соревнованиях не нужно опираться на здравый смысл при создании фич, они могут не иметь логики, но значительно помочь модели.

### Последний интересный момент в SHAP

In [ ]:
def plot_shap(row_to_show):
    row_to_show = row_to_show
    data_for_prediction = X_train.iloc[[row_to_show]]  # use 1 row of data here. Could use multiple rows if desired
    data_for_prediction_array = data_for_prediction.values.reshape(1, -1)


    model.predict_proba(data_for_prediction_array)
    print(model.predict_proba(data_for_prediction_array))
    print(train.iloc[[row_to_show]].TARGET)
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(data_for_prediction)

    shap.initjs()
    return shap.force_plot(explainer.expected_value[1], shap_values[1], data_for_prediction)
plot_shap(7)

Как это интерпретировать?

Признаки, которые увеличивают популярность, указаны синим, а которые уменьшают - розовым. Так можно посмотреть на каждое из объявлений и понять, почему модель решила дать этому элементу тот или иной класс популярности.

Чем левее от base_value значение f(x), тем более популярным считается это объявление. f(x) - вывод модели по данному объявлению, base_value - средний вывод модели по переданному нами набору обучающих данных. Справа непопулярные объявления, слева популярные.

In [ ]:
plot_shap(5)

Значение f(x) у классов medium и high часто одинаковое - эти классы отличаются совсем чуть-чуть по значению f(x), но отличаются признаки, которые указывают на один из этих классов.

In [ ]:
plot_shap(3)

Но в классе low предсказанное значение f(x) всегда намного правее